In [4]:
import pandas as pd
import numpy as np

### Remote I/O

In [5]:
data_model_output_file = '../../data/processed/data_model_output.csv'
airport_routes_shapes_file = '../../data/external/gtfs-snippets/airport-routes.csv'
output_data_file = '../../data/interim/origins-close-to-transit.csv'

### Data Reads

In [6]:
input_data_df = pd.read_csv(data_model_output_file)
airport_routes_df = pd.read_csv(airport_routes_shapes_file)

C:\Users\mwe\AppData\Local\Temp\ipykernel_22512\1623197710.py:1: DtypeWarning: Columns (1,13,14,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,47,48,54,55,59,65,69,74,77,78,80,82,83,86,93,95,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,132,142,147,150,151,152,155,156,157,167,169,177,179,196,197,198,201,210,211,216,223,237,239,240,241,242,243,244,246,248,249,273,283,284,285,288,291,295,296,297,299,300,335,337,338,340,411,414,435) have mixed types. Specify dtype option on import or set low_memory=False.
  input_data_df = pd.read_csv(data_model_output_file)


In [7]:
airport_routes_df.head()

,shape_id,lat,lon,shape_pt_sequence,shape_dist_traveled
0,S2_992_9_60,32.715870,-117.152870,10001,0.000
1,S2_992_9_60,32.715740,-117.152870,10002,0.009
2,S2_992_9_60,32.715733,-117.153786,10003,0.063
3,S2_992_9_60,32.715733,-117.154060,10004,0.079
4,S2_992_9_60,32.715733,-117.154060,20001,0.079


In [8]:
input_data_df.filter(like='inbound', axis=1)

,inbound_or_outbound,inbound_or_outbound_label
0,1.0,INBOUND_TO_AIRPORT
1,1.0,INBOUND_TO_AIRPORT
2,1.0,INBOUND_TO_AIRPORT
3,1.0,INBOUND_TO_AIRPORT
4,1.0,INBOUND_TO_AIRPORT
...,...,...
9887,2.0,OUTBOUND_FROM_AIRPORT
9888,2.0,OUTBOUND_FROM_AIRPORT
9889,2.0,OUTBOUND_FROM_AIRPORT
9890,2.0,OUTBOUND_FROM_AIRPORT


### Reductions

In [9]:
data_df = input_data_df[[
    "unique_id",
    "origin_latitude",
    "origin_longitude",
    "main_mode_label",
    "marketsegment_label",
    "inbound_or_outbound_label",
    "weight_departing_and_arriving"
]]
data_df = data_df[data_df["origin_latitude"].notna()]

In [10]:
def haversine(lat1, lon1, lat2, lon2):
    radius = 6371
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance_in_kilometers = radius * c
    return distance_in_kilometers


In [11]:
min_distances = []
for index, row in data_df.iterrows():
    lat1, lon1 = row['origin_latitude'], row['origin_longitude']
    distances = [haversine(lat1, lon1, row2['lat'], row2['lon']) for index2, row2 in airport_routes_df.iterrows()]
    min_distance = min(distances)
    min_distances.append(min_distance)

# Create a new DataFrame with the minimum distances
distances_df = pd.DataFrame({'unique_id': data_df['unique_id'], 'distance_in_kilometers': min_distances})


In [12]:
distances_df.head()

,unique_id,distance_in_kilometers
0,1,5.450920
1,2,1.211928
2,3,3.447726
3,4,0.421076
4,5,1.345806


In [13]:
data_df = pd.merge(data_df, distances_df, on="unique_id")
data_df.head()

,unique_id,origin_latitude,origin_longitude,main_mode_label,marketsegment_label,inbound_or_outbound_label,weight_departing_and_arriving,distance_in_kilometers
0,1,32.816714,-117.176898,DROVE_ALONE_AND_PARKED,EMPLOYEE,INBOUND_TO_AIRPORT,10.840259,5.450920
1,2,32.639943,-117.085774,MTS_ROUTE_992,EMPLOYEE,INBOUND_TO_AIRPORT,6.487856,1.211928
2,3,32.743009,-117.131699,DROVE_ALONE_AND_PARKED,EMPLOYEE,INBOUND_TO_AIRPORT,10.840259,3.447726
3,4,32.706752,-117.148810,MTS_ROUTE_992,EMPLOYEE,INBOUND_TO_AIRPORT,6.487856,0.421076
4,5,32.678108,-117.099196,MTS_ROUTE_992,EMPLOYEE,INBOUND_TO_AIRPORT,6.487856,1.345806


In [14]:
data_df.head()

,unique_id,origin_latitude,origin_longitude,main_mode_label,marketsegment_label,inbound_or_outbound_label,weight_departing_and_arriving,distance_in_kilometers
0,1,32.816714,-117.176898,DROVE_ALONE_AND_PARKED,EMPLOYEE,INBOUND_TO_AIRPORT,10.840259,5.450920
1,2,32.639943,-117.085774,MTS_ROUTE_992,EMPLOYEE,INBOUND_TO_AIRPORT,6.487856,1.211928
2,3,32.743009,-117.131699,DROVE_ALONE_AND_PARKED,EMPLOYEE,INBOUND_TO_AIRPORT,10.840259,3.447726
3,4,32.706752,-117.148810,MTS_ROUTE_992,EMPLOYEE,INBOUND_TO_AIRPORT,6.487856,0.421076
4,5,32.678108,-117.099196,MTS_ROUTE_992,EMPLOYEE,INBOUND_TO_AIRPORT,6.487856,1.345806


In [15]:
data_df.to_csv(output_data_file, index=False)